In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

import statsmodels.formula.api as smf


import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

We start by loading the previously created dataframe with the answers to the questions we asked gpt. We also read the 20 questions, which are stored in `questions` below.

In [ ]:
romance_with_binary = pd.read_csv('./Data/Preprocessed/romances.with.binary.tsv', delimiter='\t')
romance_with_binary.head(3)

In [ ]:
f = open('Data/trope_questions.txt', 'r')

line = f.readline()
questions = []

while len(line)!=0:
    questions.append(line.strip())
    line = f.readline()
    
questions = questions[1:-1]

# A) Preprocessing

In [ ]:
gpt_answers = romance_with_binary[['movie_id', 'binary_answers']].copy()
gpt_answers['response_length'] = gpt_answers['binary_answers'].apply(lambda ans: len(ans))
#gpt_answers.head(3)

In [ ]:
bits = {'0', '1', '2'}
gpt_answers['binary_answers'] = gpt_answers['binary_answers'].apply(lambda b: ''.join(c for c in b if c in bits))
gpt_answers['response_length'] = gpt_answers['binary_answers'].apply(lambda ans: len(ans))

In [ ]:
gpt_answers['binary_answers'] = gpt_answers['binary_answers'].apply(lambda b: b if len(b)==20 else '2'*20)
gpt_answers.drop(columns=['response_length'], inplace=True)
gpt_answers.head(5)

In [ ]:
for q_nb, question in enumerate(questions):
    gpt_answers[f'q_{q_nb}'] = gpt_answers['binary_answers'].apply(lambda b: int(b[q_nb]))
    
gpt_answers.head(5)

And let's plot how many answers to the questions were negative, positive and unknown for each question.

In [ ]:
def plot(df):
    fig, ax = plt.subplots(4, 5, figsize= (20,8), sharey = True, sharex = True)

    for i in range(20):
        sbplt = ax[i%4, math.floor(i/4)]
        sbplt.hist(df[f'q_{i}'])
        sbplt.set_title(f'Question {i}')

    fig.tight_layout()
    fig.suptitle('Histogram of answers (0, 1, and 2) for each question', y=1.05, fontsize=20)
    plt.show()

#plot(gpt_answers)

In [ ]:
def plot_pie_charts(df):
    fig, ax = plt.subplots(nrows = 4, ncols = 5, figsize = (12,10))

    for r in range(4):
        for c in range(5):
            ax[r,c].pie(df[f'q_{r*5+c}'].value_counts(),
                        labels = df[f'q_{r*5+c}'].dropna().unique(),
                        autopct=lambda p: '{:.1f}%'.format(p),
                        startangle=90, shadow=False)
            ax[r,c].set_title(f'Question {r*5+c}')
    
    fig.suptitle('Pie charts of percentage of answers (0, 1, and 2) for each question', y=1.01, fontsize=15)
    fig.tight_layout()
            
#plot_pie_charts(gpt_answers)

In [ ]:
relevant_answers = gpt_answers[gpt_answers['binary_answers']!='2'*20].copy()

print(f"There are {relevant_answers['binary_answers'].count()-relevant_answers['binary_answers'].str.contains('1').sum()} movies that do not even contain a 1")

As seen above, there are around 600 movies that have no positive answer at all. Some examples of these movies are Spirit, Harry Potter movies... Note that these movies should not be considered romance movies in the first place ! It is thus good that GPT outputs a `00000000000000000000` when analyzing them. 

In fact, this gives us a rationale to trim even more the dataset : if a movie does not contain any of the 20 clichés, is it in fact a romance movie ? It seems that most of the all `0`'s strings are in fact from movies that are not romance centered at all. We thus choose an (arbitrary) threshold to consider a movie `Romance` : it should contain at least 3 positive answers. 

In [ ]:
# First remove any string consisting of only 2s
#df = gpt_answers[gpt_answers['binary_answers']!='2'*20].copy()

# Define the questions list, ie all columns
questions_list = ['q_0', 'q_1', 'q_2', 'q_3', 'q_4', 'q_5', 'q_6', 'q_7', 'q_8', 'q_9', 'q_10', 'q_11', 'q_12', 'q_13', 'q_14', 'q_15', 'q_16', 'q_17', 'q_18', 'q_19']

# We keep only the strings with at least a 1. 
# This way we remove the all 0s strings, the all 2s strings, and the mixed strings.
data = gpt_answers[gpt_answers['binary_answers'].str.contains('1')].copy()

filenames = data['movie_id']
filenames = filenames.apply(lambda x : str(x) + '.xml.gz').values

# 4380 different binary answers
print(f"There are {len(gpt_answers['binary_answers'].unique())} different answers.")

# Remove all the entries that (somehow) have more than 20 positive answers, and then plot the histogram of #1s.
data['total'] = data[questions_list].sum(axis = 1)
data = data[21 > data['total']]
plt.hist(data['total'], bins = 20)
plt.show()

We should expect, after keeping only movies with 3 positive answers or more, to have around 4500 movies.

In [ ]:
# Keep only entries with at least 3 positive answers: this captures the "real" romance movies.
data = data[data['total'] > 0]
display(data)

Now that we refined our dataset, we only have $4653$ movies now. Let's plot each question's statistics !

In [ ]:
plot_pie_charts(data)

In [ ]:
col_to_keep = ['movie_id', 'movie_release', 'binary_answers']
for i in range(len(questions)):
    col_to_keep.append(f'q_{i}')
    
# Perform a right merge : we only care about the relevant answers
gpt_year = romance_with_binary.merge(data, on=['movie_id', 'binary_answers'], how='right')[col_to_keep]
gpt_year.replace(2, np.NaN, inplace=True)

At this point, we can wonder : is there a clear trend as to how clichés evolve over time ? As an example, could we say that modern movies contain less weddings stopped at the altar ?

In [ ]:
def plot_answer_by_year(df): 
    fig, ax = plt.subplots(4, 5, figsize= (20,8), sharey = True, sharex = True)

    for i in range(20):
        ax[i%4, math.floor(i/4)].set_title(f'Question {i}')
        sns.lineplot(df, x='movie_release', y=f'q_{i}', ax=ax[i%4, math.floor(i/4)])

    fig.tight_layout()
    plt.show()
    
#plot_answer_by_year(gpt_year)
#plot_answer_by_year(gpt_year[gpt_year['movie_release']>1920])

In [ ]:
#plot_answer_by_year(gpt_year[gpt_year['movie_release']>1920])

# B) Analysis of single questions

In [ ]:
## When sophie gives tsv : just import it as df

In [ ]:
trimmed = data.drop(['binary_answers'], axis = 1)
trimmed = trimmed.replace(2, np.NaN)
#trimmed = trimmed.astype(pd.Int64Dtype())
df = trimmed.merge(romance_with_binary, on = 'movie_id', how = 'left')

df.drop_duplicates('movie_id', inplace=True)

# TODO : MAYBE REMOVE THIS !
#df = df[df.nb_votes > 10000]

#print(df.info())
df = df.sort_values(by='nb_votes')
plt.plot(np.log(df.nb_votes), df.rating)


In [ ]:
df_mva = df.dropna(subset = ['rating', 'nb_votes'])
df_mva['mva'] = df_mva['rating'].rolling(100).mean()
df_mva.dropna(subset = 'mva')
df_mva = df_mva.sort_values(by='nb_votes')
# moving average with a window size of 100
fig, ax = plt.subplots()

plt.plot(np.log(df_mva.nb_votes), df_mva.mva)


We want to reduce this bias! Hence, we only keep movies with more than 500 reviews.

In [ ]:
df = df[(df.nb_votes > 500)].copy()

In [ ]:
sns.lineplot(data=df, x='movie_release', y='rating')

In [ ]:
df["ratings_with_weight"] = [ v + w*1j for v,w in zip(df.rating, df.nb_votes)]
def weighted_mean(x, **kws):
    return np.sum(np.real(x) * np.imag(x)) / np.sum(np.imag(x))

def plot_error_bars(df, nr_rows, nr_cols, weights):
    
    fig, ax = plt.subplots(nr_rows, nr_cols, figsize=(nr_cols*4,nr_rows*3), squeeze=False)
    
    for r in range(nr_rows):
        for c in range(nr_cols):
            
            index = r * 5 + c
            question = 'q_' + str(index)
            
            if weights : 
                sns.barplot(data=df, x= question, y='ratings_with_weight', ax = ax[r][c], estimator = weighted_mean, errorbar=('ci', 95))
                ax[r][c].set_ylim(6.65,7.5)
            else :
                sns.barplot(data=df, x= question, y='rating', ax = ax[r][c])
                ax[r][c].set_ylim(6.2,6.9)

            ax[r][c].set_ylabel('average rating')
            ax[r][c].set_title(question)
            #ax[r,c].set_title(question)
    
    fig.suptitle('AHOUUUUUUUUUUUUU', y=1.01, fontsize=15)
    plt.tight_layout()    
    plt.show()
    
plot_error_bars(df, 4, 5, False)
#plot_error_bars(df, 4, 5, True)

In [ ]:
# This functions is to be used when analyzing pairs of questions. 
def pair_analysis(id1, id2) : 

    id1 = str(id1)
    id2 = str(id2)
    q1 = 'q_' + id1
    q2 = 'q_' + id2 
    
    df_cleaned = df.dropna(subset=[q1, q2])
    dft = df_cleaned.copy()
    dft['merged_' + id1 + id2] = list(zip(df_cleaned[q1], df_cleaned[q2]))
    
    sns.barplot(data = dft, x='merged_'+ id1 + id2,  y='rating')

pair_analysis(7, 15)
plt.ylim(6.1, 6.8)

In [ ]:
sns.lineplot(data = df, x= 'movie_release', y='rating', hue = 'q_0', palette = sns.color_palette()[:df.q_0.nunique()])

# C) Analysis of ratings

In [ ]:
df_cleaned = df.dropna(subset=questions_list)

#define predictor and response variables
X, y = df_cleaned[questions_list], df_cleaned.rating

#fit regression model
reg = LinearRegression().fit(X, y)

#print(reg.coef_)
print(f"For this regression, r-squared score is : {reg.score(X, y)}")

The model may be too simple... We should maybe try to take into account interaction terms ! 

In [ ]:
X, y = df_cleaned[questions_list].to_numpy(), df_cleaned.rating.to_numpy()

print(f"Initially: \nX has shape {X.shape}\ny has shape {y.shape}\n")

degree = 2
poly = PolynomialFeatures(interaction_only=True,include_bias = False, degree = degree)
X_interactions = poly.fit_transform(X)
print(f"After adding interactions terms up to degree {degree}: \nX has shape : {X_interactions.shape}\n")

reg = LinearRegression()
reg.fit(X_interactions,y)
print(f"The r-squared score is {reg.score(X_interactions, y)}")

cv_scores = cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation mean-squared error : {-cv_scores}')

This is bad. Whether it's degree 2,3 or 4, we either overfit or underfit... At this point we can wonder whether it's infact a problem of our model, or of the data. Let us try one last model : the random forest.

# D) Random Forest

In [ ]:
# Remove questions 14, 15 as they were observed to be rather unaccurate.
cleaned_questions_list = ['q_0', 'q_1', 'q_2', 'q_3', 'q_4', 'q_5', 'q_6', 'q_7', 'q_8', 'q_9', 'q_10', 'q_11', 'q_12', 'q_13', 'q_16', 'q_17', 'q_18', 'q_19']

In [ ]:
df_cleaned = df.dropna(subset=cleaned_questions_list)
X, y = df_cleaned[cleaned_questions_list], df_cleaned.rating

# Create an instance of a random forest regressor, using sklearn.
regressor = RandomForestRegressor(n_estimators=5, random_state=42, max_depth = 15)

# Fit the regressor with X and y data.
regressor.fit(X, y)

# Make predictions for the initial data.
predictions = regressor.predict(X)
 
# Extract MSE and R2 scores to evaluate the model.
mse = mean_squared_error(y, predictions)
print(f'Mean Squared Error: {mse}')
 
r2 = r2_score(y, predictions)
print(f'R-squared: {r2}')

This seems okay, let us perform cross-validation to verify that we indeed predict accurately !

In [ ]:
# Perform cross-validation
cv_scores = cross_val_score(regressor, X, y, cv = 5, scoring = 'neg_mean_squared_error')  # Adjust the number of folds as needed

print(f'Cross-Validation scores: {cv_scores}')

We obtain a MSE of around 1 in each case, but the R2 was high. This means we are (again) overfitting...

In [ ]:
feature_importances = regressor.feature_importances_
print('Feature Importances:', feature_importances)

Whether it is a random forest, a naive regression or a regression with interaction terms, we always do bad in the prediction. 

We could thus say there is no clear and simple model as to how movies are rated. In a sense, this is reassuring : humans are not robots, and we do not appreciate movies based solely on a yes-no flow chart concerning the plot ! There is much more to movies than just the plots :)

Also, keep in mind that although checked to be rather correct, GPT (and any question-answering model nowadays) results may be inaccurate, especially when a question expresses "feelings", or requires interpretation (e.g. "Is the ending sad").

In the end, does this mean we can not do anything using this data ? Not at all ! In fact, although there is no clear way to predict a movie's success based on its tropes, we can still analyze which tropes are positively and negatively correlated (think about the barplots above). We can even do statistical tests based on matchings, on a subset of relevant questions.

# E) Feature Selection

As seen above, many questions seem to be irrelevant. We thus focus on 6 or 7 questions, in order to perform matchings and statistical tests. That way, we ensure that we have enough movies to make a matching.

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import  LinearRegression

model =  LinearRegression()  # Choose an appropriate model
rfe = RFECV(model, step=1, cv=5, scoring='neg_mean_squared_error')
fit = rfe.fit(X, y)

selected_features = list(zip(X.columns[fit.support_], fit.estimator_.coef_)) 
score = fit.score(X, y)

selected_features.sort(key = lambda x: np.abs(x[1]))
print("Selected features/Feature importance :")
for feature in selected_features : 
    print(f"   {feature[0]} : {round(feature[1], 2)}")
    
print(f"Score : {round(score, 2)}\n")

Note that again, the score is really low. This is normal, as, as previously seen, there is no way to predict accurately ratings of the movies.

There are 9 significant features, which yields 512 bins. This is a bit too much for 4000 movies, so we choose the 7 most important questions, ie only 128 bins.

In [ ]:
selected_questions = ['q_0', 'q_3', 'q_4', 'q_18']

# Questions to remove, computes 'questions_list - selected_questions'
questions_to_remove = [x for x in questions_list if x not in selected_questions]

print(questions_to_remove)

filtered_df = df_cleaned.drop(columns = questions_to_remove)

filtered_df.head(3)

# F) Observational Study

We were unable to predict ratings of movies based on their ratings, and also unable to perform a clean regression. But this does not mean we are done ! We can still analyze whether clichés are positively or negatively correlated ! 

In [ ]:
matching = filtered_df.groupby(selected_questions).size().reset_index().rename(columns={0:'count'})

In [ ]:
def top_n_indices(data, n):
    ind = np.argpartition(data, -n)[-n:]
    ind = ind[np.argsort(data[ind])][::-1]
    return ind

print(matching.iloc[top_n_indices(matching['count'], 8)])

### 1) Question 0 : do people hate weddings stopped at the altar ? :(

In [ ]:
sns.barplot(filtered_df[['q_0', 'rating', 'nb_votes']], x='q_0', y ='nb_votes')

In [ ]:
def create_matching(questions, filtered_data):

    # Questions should be of form "1000", "0100", "0010", or "0001".
    # This is because we noticed that in any case, the biggest control group is the "0000" one
    split = list(questions)    
    q_0 = int(split[0])
    q_3 = int(split[1])
    q_4 = int(split[2])
    q_18 = int(split[3])
        
    # First, we extract the indices of all the movies with binary answers:
    # Form "0000" for the control group
    # Form "0100" for the treatment group (e.g.)
    control_indices = filtered_df[(filtered_df['q_0'] == 0) & (filtered_df['q_3'] == 0) & (filtered_df['q_4'] == 0) & (filtered_df['q_18'] == 0)].index
    treatment_indices = filtered_df[(filtered_df['q_0'] == q_0) & (filtered_df['q_3'] == q_3) & (filtered_df['q_4'] == q_4) & (filtered_df['q_18'] == q_18)].index
    total = list(control_indices) + list(treatment_indices)
    
    # We now extract the sub-dataframes from the indices.
    control = filtered_df.loc[control_indices]
    control = control.sort_values(by = 'nb_votes')

    treatment = filtered_df.loc[treatment_indices]
    treatment = treatment.sort_values(by = 'nb_votes')
    
    # This creates the columns names, useful for later.
    cols_x = list(treatment.columns + '_x')
    cols_x.remove('nb_votes_x')
    cols_y = list(treatment.columns + '_y')
    cols_y.remove('nb_votes_y')

    # Perform a merge : for each movie in the treatment group, this pairs it
    # with the movie in the control group with the closest number of votes    
    test = pd.merge_asof(treatment, control, on="nb_votes", direction="nearest")

    # After the pairing, extract the groups into two separate datasets
    treatment_matched = test[cols_x].copy()
    treatment_matched.rename(mapper = lambda name : name[:-2], axis='columns', inplace=True)
    treatment_matched = treatment_matched.merge(filtered_df[['movie_id','nb_votes']],on='movie_id', how='left')

    control_matched = test[cols_y].copy()
    control_matched.rename(mapper = lambda name : name[:-2], axis='columns', inplace=True)
    control_matched = control_matched.merge(filtered_df[['movie_id','nb_votes']],on='movie_id', how='left')
    

    matching = pd.concat((control_matched, treatment_matched))
    
    print(f"There are {len(control_matched)} control movies, and {len(treatment_matched)} treatment movies")
    #control_matched.drop_duplicates(subset='movie_id', keep='first', inplace = True)
    return matching, control_matched, treatment_matched
    

In [ ]:
features_dict = {'q_0' : '1000', 'q_3' :'0100', 'q_4' :'0010', 'q_18' : '0001'}

def get_standardized_mean_diff(treatment, control):
    return np.abs(treatment.mean()-control.mean())/np.sqrt(treatment.std()**2+control.std()**2)


def observational_regression(question) :
    
    matching, control_matched, treatment_matched = create_matching(features_dict[question], filtered_df)
    smd = get_standardized_mean_diff(control_matched['nb_votes'], treatment_matched['nb_votes'])
    print(f"SMD = {round(smd, 4)}")

    mod = smf.ols(formula='rating ~ ' + question, data=matching)
    print(mod.fit().summary())
    

In [ ]:
observational_regression('q_0')

That's good ! We found 576 pairs of movies that have the same answers on the most important questions. Moreover, we made an approximate matching based on their number of votes. This is confirmed through the SMD criterion, which is extremely low (around .0002).

This way, we reduces all the possible biases. In the end, we can confirm : as the p-value is really low, we can thus accept the hypothesis that average rating for movies with a wedding stopped at the altar and without is the same!

Conclusion : weddings stopped at the altar are BAD !